## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 6.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=4d160a43c467a679f600134303627f0984bfae00e838f6ec7d03e60408ee2535
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=65ad2c4a23bc3e76baba674e5bb38a4794d452788c1ed27f29251efaeda1a431
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
print(tf.__version__)

2.4.1


##Data Preprocessing

In [5]:
#dataset from keras
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
#train-test images
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
#scale down images from 0-255
train_images=train_images/255.0
test_images=test_images/255.0

In [9]:
#dataset image shape
train_images[0].shape

(28, 28)

In [10]:
#1 channel images
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [11]:
#building the model
#hp - hyperparameter
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [13]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 20s]
val_accuracy: 0.8629999756813049

Best val_accuracy So Far: 0.9129999876022339
Total elapsed time: 00h 01m 59s
INFO:tensorflow:Oracle triggered exit


In [15]:
model=tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        55360     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2949200   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 3,006,330
Trainable params: 3,006,330
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1165 - accuracy: 0.9576 - val_loss: 0.2644 - val_accuracy: 0.9175
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0775 - accuracy: 0.9719 - val_loss: 0.3092 - val_accuracy: 0.9128
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0506 - accuracy: 0.9824 - val_loss: 0.3403 - val_accuracy: 0.9150
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0382 - accuracy: 0.9860 - val_loss: 0.4074 - val_accuracy: 0.9070
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0313 - accuracy: 0.9888 - val_loss: 0.4590 - val_accuracy: 0.9087
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0178 - accuracy: 0.9936 - val_loss: 0.5031 - val_accuracy: 0.9130
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0151 - accuracy: 0.9950 - val_loss: 0.6027 - val_accuracy